In [ ]:
import numpy as np
from scipy import special

The cell below is the default function that parametrizes the angle as a function of time. You can create your own in a designated cell below but it must have the following characteristics:

$
\begin{cases}
\theta(t) = \pi,    &t = t_{start}\\
\theta(t) = \frac{\pi}{2},  &t = t_{1/2}\\
\theta(t) = 0,  &t = t_{final}
\end{cases}
$

Where $t_{1/2}$ is the midpoint between $t_{start}$ and $t_{final}$

The default uses a shifted error function that can be adjusted via the $\tau$ and $t_{mid}$ parameters to shift the center and how the values of the function go from $\pi$ to 0. The reason we are sweeping from $\pi$ to 0 is because we are following the trajectory of a start in the interpulse region of a TP-AGB star. Should we sweep the angle from 0 to $\pi$, the trajectory will run backwards.

In [ ]:
def t_dist(tau,t,t_mid):
    #distributes the angle as a function of time from pi to 0
    delta_t = t - t_mid
    return np.pi*(1 - special.erf(delta_t/tau))/2

This cell contains the functions that will build the trajectory based on the parameters set. The parameters are described below. 

What the method does is parametrize the stellar trajectory in $n_{n}$ vs $T_{9}$ (Neutron Number Density vs Temperature in the billions of Kelvins). The parametrization carried out is on a log-lin scale. Below, the user will supply the power of 10 (decimals are allowed) for $n_{n}$ and $T_{9}$. The method then splits the coordinate pair and fits 2 sets of 3 equations with 3 unknowns for the parameters of the ellipse to be applied in the following parametrization

$
\begin{cases}
x(t) &= a_{x}cos(\theta(t)) - b_{x}sin(\theta(t)) + x_{0}\\
y(t) &= a_{y}cos(\theta(t)) - b_{y}sin(\theta(t)) + y_{0}
\end{cases}
$

The parameters determined are $a_{x},a_{y},b_{x},b_{y},x_{0},$ and $y_{0}$. The a's and b's are the projections of the semi major and minor axes along an unknown incline while $(x_{0},y_{0})$ is the coordinate pair of the center.

Upon successful return, the method will return $x(t)$, $y(t)$, and an array of time. 

In [1]:

def var_calc(t9,nn):
    #calculates the variables required to parametrize the trajectory
    A= np.array([[-1,0,1],[1,0,1],[0,-1,1]])
    
    x = np.linalg.solve(A,t9)
    rx_cos = x[0]
    ry_sin = x[1]
    x_0 = x[2]

    x = np.linalg.solve(A,nn)
    rx_sin = x[0]
    ry_cos = x[1]
    y_0 = x[2]

    return [x_0,y_0,rx_cos,ry_sin,rx_sin,ry_cos]

def pulse(varia,t,param_list,time_func = None):
    time = np.linspace(t[0],t[1],param_list['pulse_size'])

    theta = time_func(param_list['tau'],time,t[1]/2)

    x_t = varia[2]*np.cos(theta) - varia[3]*np.sin(theta) + varia[0]
    y_t = varia[4]*np.cos(theta) - varia[5]*np.sin(theta) + varia[1]
        
    return x_t,y_t,time
    
def pulse_return(varia,t, param_list, t9,nn, time_func,t9_return = None):

    if t9_return is None: t9_return = t9[0]

    t_pulse = param_list['pulse_frac']*t[1]
    t_return_start = t_pulse*(1 + 1e-3)
    t_peak = t_pulse/2
    
    slope = (nn[1] - nn[0])/(t9[1] - t9[0])
    inter = nn[1] - slope*t9[1]

    x_t,y_t,time = pulse(varia,[t[0],t_pulse],param_list,time_func = None)

    time = np.append(time,np.linspace(t_return_start,t[1],param_list['return_size']))
    x_t = np.append(x_t,np.linspace(t9[1],t9_return),param_list['return_size'])
    y_t = np.append(y_t,slope*np.linspace(t9[1],t9_return,param_list['return_size']) + inter)
    
    return x_t,y_t,time

def pulse_return_loop(varia,t,param_list,t9,nn,time_func = None):

    slope_vert = (nn[2] - nn[0])/(t9[2] - t9[0])
    slope_hori = (nn[1] - nn[0])/(t9[1] - t9[0])

    inter_vert = nn[0] - slope_vert*t9[0]
    inter_hori = nn[0] - slope_hori*t9[0]

    dist_vert = ((nn[2] - nn[0])**2 + (t9[2] - t9[0])**2)**(1/2)
    dist_hori = ((nn[1] - nn[0])**2 + (t9[1] - t9[0])**2)**(1/2)

    t_loop = t[1]/param_list['n_loops']

    t9_return = t9[0] + param_list['return_scale']*dist_hori/(1 + slope_hori**2)**(1/2)

    x_t,y_t,time = pulse_return(varia,[t[0],t_loop],param_list,t9,nn,time_func,t9_return)

    for i in range(1, param_list['n_loops']):
        
        nn_return = slope_hori*t9_return + inter_hori

        t9_2 = t9_return + dist_hori/(1 + slope_hori**2)**(1/2)
        nn_2 = slope_hori*t9_2 + inter_hori

        t9_3 = t9_return + (1 + param_list['peak_scale'])**(i) * dist_vert/(1 + slope_vert**2)**(1/2)
        b = nn_return - slope_vert*t9_return
        nn_3 = slope_vert*t9_3 + b

        varia = var_calc([t9_return,t9_2,t9_3],[nn_return,nn_2,nn_3])
        x_ti,y_ti,time_i = pulse_return(varia,[t[0],t_loop],param_list,[t9_return,t9_2,t9_3],[nn_return,nn_2,nn_3],time_func,t9_return)
        time = np.append(time, i*t_loop + time_i)
        t9_return = t9_return + param_list['return_scale']*dist_hori/(1 + slope_hori**2)**(1/2)
        
    return x_t,y_t,time

def traj_builder(t,t9,nn,traj,param_list,time_func = t_dist):
    #builds the trajectories and saves to file
    #t = 2x1 array of start and end time
    #t9 = 3x1 array containing start, end, and peak t9 (in that order)
    #nn = 3x1 array containing the power of 10 of the start, end, and peak of neutron number density (in that order)
    
    varia = var_calc(t9,nn)
    
    if traj == 'pulse':
        x_t,y_t,time = pulse(varia, t, param_list,time_func)
        
    elif traj == 'pulse_return':
        x_t, y_t,time = pulse_return(varia,t,param_list,t9,nn,time_func)

    elif traj == 'pulse_return_loop':
        x_t,y_t,time = pulse_return_loop(varia,t,param_list,t9,nn,time_func)

    return x_t,y_t,time

def param_builder(**kwargs):
    return kwargs

In [ ]:
#setup the parameters to build your trajectory

#temperature in billions of kelvin
t9_1 = 0.125 #start temperature
t9_2 = 0.25  #end temperature
t9_3 = 0.3   #temperature at peak n_n
####################################

#the power of 10 of neutron number density
nn_1 = 1.1  #start
nn_2 = 2    #end
nn_3 = 9    #peak
###########################################

#time
t_start = 0 #start time default time 0
t_end = 3.15e+12 #end time of the whole trajectory in seconds

#parameters for the trajectory
return_size = 15 #number of points in time for the linear return part of the trajectory
              #defaults to 15
pulse_frac = 0.95 #the amount of time per cycle allocated for the pulse 
             #defaults to 0.95
tau = 1e+12 #relaxation timescale for the error function, typically same order as t_end
pulse_size = #total number of points in the pulse
             #defaults to 50
n_loops = 35 #total number of loops in the trajectory
return_scale = 0.01 #how far from t9_1 should the new loop start. 0 returns to it
                #defaults to 0
peak_scale = 0 #scales the next pulse relative to the one before. 0 maintains peak height
              #defaults to 0

traj = 'pulse' #you have 3 choices here and they are all strings: pulse, pulse_return, pulse_return_loop
          #pulse simply creates the trajectory from A to B
          #pulse_return creates the trajectory from start to finish then returns it to the start
          #pulse_return_loop creates a trajectory that loops on itself or loops do a different starting point

#you can create your custom parametrized time function here and pass it into the trajectory builder
#the properties are listed above

#ONLY ADD THE TIME FUNCTION TO THE BUILDER IF YOU WANT TO. The commands simply pack the parameters and calls the trajectory builder
param_list = param_builder(return_size,pulse_frac,tau,pulse_size,n_loops,return_scale,peak_scale)
t = [t_start,t_end]
t9 = [t9_1,t9_2,t9_3]
nn = [nn_1,nn_2,nn_3]
x_t,y_t,time = traj_builder(t,t9,nn,traj,param_list) #if you have a custom parametrization for the angle as a function of time,
                                                     #pass it after param_list


Now that we have our parametrized trajectory, the only missing is to define a density profile and save to file

In [ ]:
#I defaulted to a constant density because it's the simplest. The user can define what they wish

rho = 1e+3*np.ones(len(time)) #constant density of 1000 g/cc as an example
temp_dens_data = np.column_stack((time,x_t,rho))

np.savetxt('temp_dens_data.txt',temp_dens_data) #you can ofcourse change the file name

#calculating the neutron mass fraction
X_n = (10**y_t)/(rho*6.0221409e+23) #mass_frac = n_n/(rho*N_A)

neut_mass_frac_v_time = np.column_stack((time,X_n))
np.savetxt('neut_mass_frac_v_time.txt',neut_mass_frac_v_time)

#trajectory data to file
trajectory_data = np.column_stack((time,x_t,10**y_t))
np.savetxt('trajectory.txt',trajectory_data)